# Group data into on and off diffraction intensities vs. delay time

This code is here so it is under git control, but it should be loaded on the Maxwell cluster instrument at the
European XFEL.  The file-paths refer to directories on Maxwell, specifically at time of writing for the experiment
with Sam Marks and team.

Import necessary libraries

In [ ]:
import sys

import matplotlib.pyplot as plt
import numpy as np
from extra.components import Scan, Scantool, OpticalLaserDelay
from extra.utils import imshow2
from extra_data import RunDirectory, open_run
from extra_data.components import LPD1M
from extra_geom import LPD_1MGeometry
from matplotlib.gridspec import GridSpec
from extra.components import XrayPulses

# sys.path.append("/gpfs/exfel/data/user/bermudei/FXE")
sys.path.append("/gpfs/exfel/exp/FXE/202501/p008015/usr/Shared/amore/")
from fxeutils.digitizers import process_digitizer, digitizer_peaks_params, digitizer_intensity
from fxeutils.common import align_tids, create_scan_object, scan_type
from fxeutils.pumpprobe_splitter import array_split_pp#, array_split_pp_new
from fxeutils.scattering import (
    diff_iq,
    fom_q,
    get_integrator,
    get_lpd_geom,
    integrate_run,
    ref_iq,
    split_iq,
)
from fxeutils.xes import scan_xes
from pathlib import Path
from damnit import Damnit
%matplotlib widget

In [ ]:
proposal = 8015     # number of the proposal
max_run = 190       # the code will try and process all scans from any numbered zero up to max_run
overwrite = False   # set to true if you want to reanalyze previously analyzed data, otherwise set to False for faster response

In [ ]:
db = Damnit(proposal)
cwd = Path().cwd()
savePath = cwd/'save_raw_integrated_Data'
not_pp_run_Path = savePath/f'not_pump_probe_run.npy'            # this file contains IDs of runs that cannot be read fas delay-time runs
analyzed_pp_run_Path = savePath/f'analyzed_pump_probe_run.npy'  # this file contains IDs of runs that have been previously analyzed.

In [ ]:
not_pump_probe_run = np.load(not_pp_run_Path)
analyzed_pumped_probe_run = np.load(analyzed_pp_run_Path)
for run_number in np.arange(max_run+1):
    if run_number not in not_pump_probe_run:
        if run_number in analyzed_pumped_probe_run and not overwrite:
            print(f'skipping run {run_number} not overwriting previous analyzed scan')
            continue
        else:
            print(f'working on run {run_number}')
            try: 
                run = open_run(proposal, run_number, data="all")
            except FileNotFoundError:
                print(f'run {run_number} not found')
                not_pump_probe_run = np.append(not_pump_probe_run,[run_number])
                continue
            try:
                myvar = db[run_number, "Iq_lpd"]
                az_intd = myvar.read()
            except KeyError:
                print(f'run {run_number} has no Iq_lpd data, dark image?')
                not_pump_probe_run = np.append(not_pump_probe_run,run_number)
                continue
            sorted_azint = split_iq(run, az_intd, norm=False)
            iq_on = sorted_azint["pumped"]   
            iq_off = sorted_azint["unpumped"]
            iq_diff = sorted_azint["diff"]
            q= az_intd["q"]
            try:
                scan_motor = scan_type(run)
                scan = create_scan_object(run, scan_motor)
            except KeyError:
                print(f'run {run_number} generate KeyError, no pump-probe delay?')
                not_pump_probe_run = np.append(not_pump_probe_run,run_number)
                continue
            try:
                if scan_motor:
                    scan_2d_diff = scan_xes(run, scan_motor, iq_diff.squeeze())
                    scan_2d_on = scan_xes(run, scan_motor, iq_on.squeeze())
                    scan_2d_off = scan_xes(run, scan_motor, iq_off.squeeze())
                else:
                    print(f'run {run_number} has no motor')
                    not_pump_probe_run = np.append(not_pump_probe_run,run_number)
                    continue
            except ValueError:
                print(f'run {run_number} generated ValueError in scan_xes')
                not_pump_probe_run = np.append(not_pump_probe_run,run_number)
                continue
            str_run_number = str(run_number).zfill(4)
            onPath = savePath/f'run{str_run_number}_delay_intensity_on.npy'
            offPath = savePath/f'run{str_run_number}_delay_intensity_off.npy'
            delayPath = savePath/f'run{str_run_number}_delay_positions.npy'
            qPath = savePath/f'run{str_run_number}_q_values.npy'
            np.save(onPath, scan_2d_on)
            np.save(offPath, scan_2d_off)
            np.save(delayPath, scan.positions)
            np.save(qPath, iq_on["q"].values)
            analyzed_pumped_probe_run = np.append(analyzed_pumped_probe_run,run_number)
    else:
        print(f'skipping run {run_number} not a pump probe scan')
np.save(not_pp_run_Path, not_pump_probe_run)
np.save(analyzed_pp_run_Path, analyzed_pumped_probe_run)

In [ ]:
print(not_pump_probe_run)

In [ ]:
fig, ax1 = plt.subplots(figsize=(9, 7))

ax1.plot(az_intd.q, az_intd.mean(dim=("trainId", "pulseId")))
ax1.set_xlabel("q [A^-1]")
ax1.set_ylabel("Intensity [arb. u]")
ax1.set_title("Mean I(q) over all trains and pulses")
ax1.grid()

In [ ]:
fig = plt.figure(figsize=(9, 7))
gs = GridSpec(2, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

ax1.plot(az_intd.q, az_intd.mean(dim=("trainId", "pulseId")))
ax1.set_xlabel("q [A^-1]")
ax1.set_ylabel("Intensity [arb. u]")
ax1.set_title("Mean I(q) over all trains and pulses")
ax1.grid()

pulse_mean = az_intd.mean("trainId")
im = imshow2(
    pulse_mean,
    ax=ax2,
    aspect="auto",
    extent=[az_intd.q.min(), az_intd.q.max(), len(az_intd.pulseId), 0],
)
ax2.set_xlabel("q [A^-1]")
ax2.set_ylabel("Pulse ID")
ax2.set_title("I(q) per-pulse (averaged over trains)")
fig.colorbar(im, ax=ax2)

train_mean = az_intd.mean("pulseId")
im = imshow2(
    train_mean,
    ax=ax3,
    aspect="auto",
    extent=[az_intd.q.min(), az_intd.q.max(), len(az_intd.trainId), 0],
)
ax3.set_xlabel("q [A^-1]")
ax3.set_ylabel("Train")
ax3.set_title("I(q) per-train (averaged over pulses)")
fig.colorbar(im, ax=ax3)

fig.tight_layout()

In [ ]:
sorted_azint_norm_sq = split_iq(run, az_intd, norm=True)

In [ ]:
sorted_azint_norm_iq = split_iq(run, az_intd, norm=True, q_min=0.1, q_max=11)

In [ ]:
sorted_azint_norm_i0 = split_iq(run, az_intd, norm=True, digitizer='2C_raw')

In [ ]:
digi=run.alias['2C_raw']
I0=process_digitizer(run,digi,-40,120,-500,-150,0,3000)[0]
I0_nw =process_digitizer(run,digi,-5,5,-500,-150,0,3000)[0]

In [ ]:
fig, ax = plt.subplots(figsize=(9, 7))

ax.plot(az_intd["q"], iq_on.mean(dim=["trainId", "pulseId"]), label="laser on")
ax.plot(az_intd["q"], iq_off.mean(dim=["trainId", "pulseId"]), label="laser off")
ax.set_xlabel("q $(A^{-1})$")
ax.set_ylabel("S(q)")
ax.set_title("S(q) averaged over all trains")
ax.grid()
ax2 = ax.twinx()
ax2.plot(
    az_intd["q"],
    iq_diff.mean(dim=["trainId", "pulseId"]),
    label="difference",
    color="green",
)

ax2.set_ylabel("S(q) difference")
ax.legend()
ax2.legend()